In [1]:
# import all necessary packages
import pandas as pd

In [2]:
# define the path of the dataset
mnist_path = "/usr/scratch/badile31/msc22f11/msc22f11/PlayGround/mnist_train.csv"
# read it in as a pandas dataframe
mnist_train_df = pd.read_csv(mnist_path)
mnist_train_df

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59997,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# define how many rows to read in
n_rows = 512
# read in the first n_rows rows of the dataframe
mnist_train_n_rows_df = mnist_train_df[:n_rows]
mnist_train_n_rows_df

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
508,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
509,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
510,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# separate the dataframe into the features and the labels
mnist_train_n_rows_features_df = mnist_train_n_rows_df.drop(["label"], axis=1)
mnist_train_n_rows_labels_df = mnist_train_n_rows_df["label"]

In [5]:
# scale the feature values to be between 0 and 1
mnist_train_n_rows_features_scaled_df = mnist_train_n_rows_features_df.div(255).astype("float16")
mnist_train_n_rows_features_scaled_df

,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,1x10,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
509,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
510,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# verify type of the elements in the dataframe
mnist_train_n_rows_features_scaled_df.dtypes

1x1      float16
1x2      float16
1x3      float16
1x4      float16
1x5      float16
          ...   
28x24    float16
28x25    float16
28x26    float16
28x27    float16
28x28    float16
Length: 784, dtype: object

In [7]:
# get the data type of the features
feature_type = str(type(mnist_train_n_rows_features_scaled_df.iloc[0][0])).split(".")[-1].split("'")[0]
print("Data type of the features: {}".format(feature_type))
# get the data type of the labels
label_type = str(type(int(mnist_train_n_rows_labels_df.iloc[0]))).split("<")[-1].split(">")[0].split("class")[-1].replace("'", "").strip()
print("Data type of the labels: {}".format(label_type))

Data type of the features: float16
Data type of the labels: int


In [8]:
# write the feature data into a binary file
for row in mnist_train_n_rows_features_scaled_df.values:
    bin_row = bytearray(row)
    f = open(str(n_rows) + '_features_' + str(feature_type), 'ab')
    f.write(bin_row)
    f.close()
print("Wrote {} rows of features to binary file: {}".format(n_rows, str(n_rows) + '_features_' + str(feature_type)))

Wrote 512 rows of features to binary file: 512_features_float16


In [9]:
# write the label data into a binary file
for row in mnist_train_n_rows_labels_df.values:
    bin_row = int(row).to_bytes(8, byteorder='little')
    f = open(str(n_rows) + '_labels_' + str(label_type), 'ab')
    f.write(bin_row)
    f.close()
print("Wrote {} rows of labels to binary file: {}".format(n_rows, str(n_rows) + '_labels_' + str(label_type)))

Wrote 512 rows of labels to binary file: 512_labels_int


In [30]:
# try alternative file mode for writing binary files for labels
for row in mnist_train_n_rows_labels_df[:n_rows]:
    bin_row = int(row).to_bytes(8, byteorder='little')
    # print the binary representation of the label as 32 bit number
    print(format(int.from_bytes(bin_row, byteorder='little'), '032b'))
    f = open('test_labels_bin8', 'ab')
    f.write(bin_row)
    f.close()

00000000000000000000000000000101
00000000000000000000000000000000
00000000000000000000000000000100
00000000000000000000000000000001
00000000000000000000000000001001
00000000000000000000000000000010
00000000000000000000000000000001
00000000000000000000000000000011
00000000000000000000000000000001
00000000000000000000000000000100
00000000000000000000000000000011
00000000000000000000000000000101
00000000000000000000000000000011
00000000000000000000000000000110
00000000000000000000000000000001
00000000000000000000000000000111
00000000000000000000000000000010
00000000000000000000000000001000
00000000000000000000000000000110
00000000000000000000000000001001


In [10]:
# print values of first row where values are not 0
for i in range(len(mnist_train_n_rows_features_scaled_df.iloc[0])):
    if mnist_train_n_rows_features_scaled_df.iloc[0][i] != 0:
        print("Value at index {} is {}".format(i, mnist_train_n_rows_features_scaled_df.iloc[0][i]))

Value at index 152 is 0.0117647061124444
Value at index 153 is 0.07058823853731155
Value at index 154 is 0.07058823853731155
Value at index 155 is 0.07058823853731155
Value at index 156 is 0.4941176474094391
Value at index 157 is 0.5333333611488342
Value at index 158 is 0.686274528503418
Value at index 159 is 0.10196078568696976
Value at index 160 is 0.6509804129600525
Value at index 161 is 1.0
Value at index 162 is 0.9686274528503418
Value at index 163 is 0.49803921580314636
Value at index 176 is 0.11764705926179886
Value at index 177 is 0.1411764770746231
Value at index 178 is 0.3686274588108063
Value at index 179 is 0.6039215922355652
Value at index 180 is 0.6666666865348816
Value at index 181 is 0.9921568632125854
Value at index 182 is 0.9921568632125854
Value at index 183 is 0.9921568632125854
Value at index 184 is 0.9921568632125854
Value at index 185 is 0.9921568632125854
Value at index 186 is 0.8823529481887817
Value at index 187 is 0.6745098233222961
Value at index 188 is 0.99

In [16]:
# print values of first row where values are not 0
for i in range(len(mnist_train_n_rows_features_scaled_df.iloc[1])):
    if mnist_train_n_rows_features_scaled_df.iloc[1][i] != 0:
        print("Value at index {} is {}".format(i, mnist_train_n_rows_features_scaled_df.iloc[1][i]))

Value at index 127 is 0.20000000298023224
Value at index 128 is 0.6235294342041016
Value at index 129 is 0.9921568632125854
Value at index 130 is 0.6235294342041016
Value at index 131 is 0.19607843458652496
Value at index 154 is 0.1882352977991104
Value at index 155 is 0.9333333373069763
Value at index 156 is 0.9882352948188782
Value at index 157 is 0.9882352948188782
Value at index 158 is 0.9882352948188782
Value at index 159 is 0.929411768913269
Value at index 181 is 0.21176470816135406
Value at index 182 is 0.8901960849761963
Value at index 183 is 0.9921568632125854
Value at index 184 is 0.9882352948188782
Value at index 185 is 0.9372549057006836
Value at index 186 is 0.9137254953384399
Value at index 187 is 0.9882352948188782
Value at index 188 is 0.2235294133424759
Value at index 189 is 0.0235294122248888
Value at index 207 is 0.03921568766236305
Value at index 208 is 0.23529411852359772
Value at index 209 is 0.8784313797950745
Value at index 210 is 0.9882352948188782
Value at ind

In [43]:
mnist_train_n_rows_features_scaled_df.values[261]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.3882353 , 0.99215686, 0.5686275 , 0.     